<a href="https://colab.research.google.com/github/karimmohraz/colab/blob/main/T5_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

Check GPU

In [ ]:
!nvidia-smi

In [4]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [8]:
# specifiy dataset and column names
path = './data/news_summary.csv'
summary_col = 'text'
fulltext_col = 'ctext'

config = {
    'TRAIN_BATCH_SIZE': 4,     # input batch size for training (default: 64)
    'VALID_BATCH_SIZE': 100,   # input batch size for testing (default: 1000)
    'TRAIN_EPOCHS' : 3,        # number of epochs to train (default: 10)
    'LEARNING_RATE' : 1e-4,    # learning rate (default: 0.01)
    'SEED' : 42,               # random seed (default: 42)
    'MAX_LEN' : 512,
    'SUMMARY_LEN' : 150 
    }

In [5]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len

    def __len__(self):
        return len(self.data.summary)

    def __getitem__(self, index):
        fulltext = str(self.data.fulltext[index])
        fulltext = ' '.join(fulltext.split())

        summary = str(self.data.summary[index])
        summary = ' '.join(summary.split())

        source = self.tokenizer.batch_encode_plus([fulltext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([summary], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [6]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we enumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        if _%50==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [7]:
def validate(tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [9]:
# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(config['SEED']) # pytorch random seed
np.random.seed(config['SEED']) # numpy random seed
torch.backends.cudnn.deterministic = True

# tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [13]:
# Importing and Pre-Processing the domain data
df = pd.read_csv(path ,encoding='latin-1')

# Selecting the needed columns only and renaming to summary & fulltext.
df = df[[summary_col, fulltext_col]]
df = df.rename({summary_col: 'summary', fulltext_col: 'fulltext'}, axis='columns')
# Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
df.fulltext = 'summarize: ' + df.fulltext
# df = df.sample(frac=0.1)
print(df.head())

                                             summary                                           fulltext
0  The Administration of Union Territory Daman an...  summarize: The Daman and Diu administration on...
1  Malaika Arora slammed an Instagram user who tr...  summarize: From her special numbers to TV?appe...
2  The Indira Gandhi Institute of Medical Science...  summarize: The Indira Gandhi Institute of Medi...
3  Lashkar-e-Taiba's Kashmir commander Abu Dujana...  summarize: Lashkar-e-Taiba's Kashmir commander...
4  Hotels in Maharashtra will train their staff t...  summarize: Hotels in Mumbai and other Indian c...


In [14]:
# Creation of Dataset and Dataloader
# Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state = config['SEED'])
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

# Creating the Training and Validation dataset for further creation of Dataloader
training_set = CustomDataset(train_dataset, tokenizer, config['MAX_LEN'], config['SUMMARY_LEN'])
val_set = CustomDataset(val_dataset, tokenizer, config['MAX_LEN'], config['SUMMARY_LEN'])

# Defining the parameters for creation of dataloaders
train_params = {
    'batch_size': config['TRAIN_BATCH_SIZE'],
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config['VALID_BATCH_SIZE'],
    'shuffle': False,
    'num_workers': 0
    }

# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


In [ ]:
# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=config['LEARNING_RATE'])

# Training loop
print('Initiating Fine-Tuning for the model on our dataset')

for epoch in range(config['TRAIN_EPOCHS']):
  train(epoch, tokenizer, model, device, training_loader, optimizer)

torch.save(model.state_dict(), './data/t5_summary_state_dict.pt')

# Validation loop and saving the resulting file with predictions and acutals in a dataframe.
# Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
predictions, actuals = validate(tokenizer, model, device, val_loader)
final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
final_df.to_csv('./data/predictions.csv')
print('Output Files generated for review')
print(final_df.head())

Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/modeling_t5.py:1146: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


Epoch: 0, Loss:  7.16775369644165
Epoch: 0, Loss:  1.5576294660568237
Epoch: 0, Loss:  2.176754951477051
Epoch: 0, Loss:  1.914730191230774
Epoch: 0, Loss:  2.052884340286255
Epoch: 0, Loss:  1.3532397747039795
Epoch: 0, Loss:  2.118084669113159
Epoch: 0, Loss:  2.407341718673706
Epoch: 0, Loss:  2.0403265953063965
Epoch: 0, Loss:  1.359547734260559
Epoch: 0, Loss:  2.276465654373169
Epoch: 0, Loss:  1.4711371660232544
Epoch: 0, Loss:  2.189418315887451
Epoch: 0, Loss:  1.9185739755630493
Epoch: 0, Loss:  2.204874277114868
Epoch: 0, Loss:  1.62185537815094
Epoch: 0, Loss:  2.595926523208618
Epoch: 0, Loss:  1.535740613937378
Epoch: 0, Loss:  1.7352827787399292
Epoch: 1, Loss:  2.060634136199951
Epoch: 1, Loss:  1.8844225406646729
Epoch: 1, Loss:  1.2498791217803955
Epoch: 1, Loss:  1.194539189338684
